In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
produc = pd.read_csv("baltic_energy_type.csv")

In [ ]:
# correlation of consumption and production over time in estonia

In [ ]:
consump["DateUTC"] = pd.to_datetime(consump["DateUTC"], format="%d-%m-%Y")

In [ ]:
produc = pd.read_csv("baltic_energy_type.csv")

In [ ]:
produc

In [ ]:
# Filter for oil shale production
oil_shale_produc = produc[produc["Production Type"] == "Fossil Oil shale"]

# Filter for all other production types
else_produc = produc[produc["Production Type"] != "Fossil Oil shale"]

In [ ]:
else_produc

In [ ]:
else_produc.groupby("MTU (CET/CEST)").sum(numeric_only=True)

In [ ]:
else_produc["MTU (CET/CEST)"] = pd.to_datetime(
    else_produc["MTU (CET/CEST)"], format="%d/%m/%Y"
)
oil_shale_produc["MTU (CET/CEST)"] = pd.to_datetime(
    oil_shale_produc["MTU (CET/CEST)"], format="%d/%m/%Y"
)

In [ ]:
consump

In [ ]:
# Merge consump with oil_shale_produc
oil_shale_merged = pd.merge(
    consump.rename(columns={"CountryCode": "Country", "DateUTC": "Date"}),
    oil_shale_produc.rename(columns={"Area": "Country", "MTU (CET/CEST)": "Date"}),
    on=["Country", "Date"],
    how="inner",
)

# Merge consump with else_produc
non_oil_shale_merged = pd.merge(
    consump.rename(columns={"CountryCode": "Country", "DateUTC": "Date"}),
    else_produc.rename(columns={"Area": "Country", "MTU (CET/CEST)": "Date"}),
    on=["Country", "Date"],
    how="inner",
)

In [ ]:
oil_shale_merged

In [ ]:
oil_shale_merged["lagged_production"] = oil_shale_merged["Generation (MW)"].shift(1)

In [ ]:
oil_shale_merged["lagged_consumption"] = oil_shale_merged["Value"].shift(1)

In [ ]:
non_oil_shale_merged["lagged_production"] = non_oil_shale_merged[
    "Generation (MW)"
].shift(1)

In [ ]:
non_oil_shale_merged["lagged_consumption"] = non_oil_shale_merged["Value"].shift(1)

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for consumption vs production
plt.figure(figsize=(10, 6))
plt.scatter(
    oil_shale_merged["Value"],
    oil_shale_merged["lagged_production"],
    alpha=0.5,
    color="blue",
)
plt.xlabel("Consumption (MW)")
plt.ylabel("Production (MW)")
plt.title("Scatter Plot of Consumption vs Oil Shale Production")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
oil_shale_merged

In [ ]:
non_oil_shale_merged = non_oil_shale_merged.groupby("Date").sum(numeric_only=True)

In [ ]:
non_oil_shale_merged = non_oil_shale_merged.reset_index()

In [ ]:
non_oil_shale_merged.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
non_oil_shale_merged["Value"] = non_oil_shale_merged["Date"].map(
    oil_shale_merged.set_index("Date")["Value"]
)

In [ ]:
non_oil_shale_merged

In [ ]:
oil_shale_merged

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for consumption vs production
plt.figure(figsize=(10, 6))
plt.scatter(
    non_oil_shale_merged["Value"],
    non_oil_shale_merged["Generation (MW)"],
    alpha=0.5,
    color="blue",
)
plt.xlabel("Consumption (MW)")
plt.ylabel("Production (MW)")
plt.title("Scatter Plot of Consumption vs Non Oil Shale Production")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
oil_shale_merged

In [ ]:
# Convert the 'MTU (CET/CEST)' column to datetime to extract the date
produc["Date"] = pd.to_datetime(produc["MTU (CET/CEST)"], format="%d/%m/%Y").dt.date

# Group by 'Date' and 'Area' and sum the 'Generation (MW)'
# daily_production = (
#     produc.groupby(["Date", "Area"])["Generation (MW)"].sum().reset_index()
# )

# Rename columns for clarity
produc.rename(
    columns={"Area": "Country", "Generation (MW)": "Production (MW)"},
    inplace=True,
)

In [ ]:
daily_production = produc

In [ ]:
daily_production["Date"] = pd.to_datetime(daily_production["Date"])

In [ ]:
consump = consump.rename(
    columns={"DateUTC": "Date", "CountryCode": "Country", "Value": "Consumption (MW)"}
)
consump.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
merged_data = pd.merge(consump, daily_production, on=["Date", "Country"], how="inner")
merged_data

In [ ]:
import matplotlib.pyplot as plt

# Filter data for Estonia
estonia_data = merged_data[merged_data["Country"] == "Estonia"]
# estonia_data = estonia_data[merged_data["Production Type"] != "Fossil Oil shale"]
# Get unique production types
production_types = estonia_data["Production Type"].unique()

# Create the plot
plt.figure(figsize=(14, 8))

# Plot consumption
plt.plot(
    estonia_data["Date"],
    estonia_data["Consumption (MW)"],
    label="Consumption (MW)",
    color="black",
    linewidth=2,
)

# Plot production by type
for prod_type in production_types:
    prod_data = estonia_data[estonia_data["Production Type"] == prod_type]
    plt.plot(
        prod_data["Date"],
        prod_data["Production (MW)"],
        label=f"Production ({prod_type})",
    )

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Consumption and Production by Type in Estonia Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
estonia_df = merged_data[merged_data["Country"] == "Estonia"]
estonia_df = estonia_df[estonia_df["Production Type"] != "Fossil Oil shale"]

In [ ]:
estonia_df

In [ ]:
estonia_df.drop(["Country"], axis=1, inplace=True)

In [ ]:
estonia_df = estonia_df.groupby("Date").sum(numeric_only=True)

In [ ]:
# Apply a rolling average to smooth the data
estonia_df["Smoothed Production (MW)"] = (
    estonia_df["Production (MW)"].rolling(window=7, center=True).mean()
)
estonia_df["Smoothed Consumption (MW)"] = (
    estonia_df["Consumption (MW)"].rolling(window=7, center=True).mean()
)

# Plot the smoothed data
plt.figure(figsize=(12, 6))
plt.plot(
    estonia_df["Date"],
    estonia_df["Smoothed Production (MW)"],
    label="Smoothed Production (MW)",
    color="red",
)
plt.plot(
    estonia_df["Date"],
    estonia_df["Smoothed Consumption (MW)"],
    label="Smoothed Consumption (MW)",
    color="blue",
)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Smoothed Consumption and Production in Estonia Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
estonia_df

In [ ]:
estonia_df = estonia_df.reset_index()

In [ ]:
estonia_df.columns

In [ ]:
estonia_df = estonia_df.drop(columns=["Consumption (MW)"])
estonia_df = estonia_df.merge(
    consump[consump["Country"] == "Estonia"][["Date", "Consumption (MW)"]],
    on="Date",
    how="left",
)

In [ ]:
estonia_df

In [ ]:
estonia_df

In [ ]:
estonia_df

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot consumption on the left y-axis
ax1.set_xlabel("Date")
ax1.set_ylabel("Consumption (MW) ", color="tab:blue")
ax1.plot(
    estonia_df["Date"],
    estonia_df["Smoothed Consumption (MW)"],
    color="tab:blue",
    label="Consumption",
)
ax1.tick_params(axis="y", labelcolor="tab:blue")

# Create a second y-axis for production
ax2 = ax1.twinx()
ax2.set_ylabel("Production (MW)", color="tab:red")
ax2.plot(
    estonia_df["Date"],
    estonia_df["Smoothed Production (MW)"],
    color="tab:red",
    label="Production",
)
ax2.tick_params(axis="y", labelcolor="tab:red")

# Title and layout
plt.title(
    "Consumption of Electricity and Production of Non-Oil-Shale Electricity in Estonia Over Time"
)
fig.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filter data for Estonia
estonia_data = merged_data[merged_data["Country"] == "Estonia"]
estonia_data = estonia_data[estonia_data["Production Type"] == "Fossil Oil Shale"]
# Plot consumption and total production
plt.figure(figsize=(12, 6))
plt.plot(
    estonia_data["Date"],
    estonia_data["Consumption (MW)"],
    label="Consumption (MW)",
    color="blue",
)
plt.plot(
    estonia_data["Date"],
    estonia_data["Total Production (MW)"],
    label="Total Production (MW)",
    color="green",
)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Consumption and Total Production in Estonia Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filter data for Estonia
estonia_data = merged_data[merged_data["Country"] == "Latvia"]

# Plot consumption and total production
plt.figure(figsize=(12, 6))
plt.plot(
    estonia_data["Date"],
    estonia_data["Consumption (MW)"],
    label="Consumption (MW)",
    color="blue",
)
plt.plot(
    estonia_data["Date"],
    estonia_data["Total Production (MW)"],
    label="Total Production (MW)",
    color="green",
)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Consumption and Total Production in Latvia Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filter data for Estonia
estonia_data = merged_data[merged_data["Country"] == "Lithuania"]

# Plot consumption and total production
plt.figure(figsize=(12, 6))
plt.plot(
    estonia_data["Date"],
    estonia_data["Consumption (MW)"],
    label="Consumption (MW)",
    color="blue",
)
plt.plot(
    estonia_data["Date"],
    estonia_data["Total Production (MW)"],
    label="Total Production (MW)",
    color="green",
)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Consumption and Total Production in Lithuania Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt


# Plot consumption vs production for each country
countries = merged_data["Country"].unique()
plt.figure(figsize=(12, 6))

for country in countries:
    country_data = merged_data[merged_data["Country"] == country]
    plt.plot(
        country_data["Date"],
        country_data["Consumption (MW)"],
        label=f"{country} Consumption",
    )
    plt.plot(
        country_data["Date"],
        country_data["Total Production (MW)"],
        label=f"{country} Production",
    )

plt.xlabel("Date")
plt.ylabel("MW")
plt.title("Consumption vs Production Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()